In [11]:
from pymongo import MongoClient
from datetime import datetime

In [2]:
client = MongoClient()
anycast = client.anycast_monitoring

In [9]:
m_root = anycast['m_root']

for item in m_root.find({'$where': 'function() { if(this.path4.length == 9) {return true;} else { return false; } }'}):
    print('[{}] {}: {}'.format(item['timestamp'], item['peer'], item['path4']))

[1246406400] 196613: [196613, 1125, 1103, 20965, 11537, 22388, 7660, 2500, 7500]
[1249084800] 196613: [196613, 1125, 1103, 20965, 11537, 22388, 7660, 2500, 7500]
[1251763200] 196613: [196613, 1125, 1103, 20965, 11537, 22388, 7660, 2500, 7500]
[1254355200] 196613: [196613, 1125, 1103, 20965, 11537, 22388, 7660, 2500, 7500]
[1257033600] 196613: [196613, 1125, 1103, 20965, 11537, 22388, 7660, 2500, 7500]
[1277942400] 196613: [196613, 1125, 1103, 20965, 11537, 22388, 7660, 2500, 7500]
[1280620800] 196613: [196613, 1125, 1103, 20965, 11537, 22388, 7660, 2500, 7500]


## which peer have large path length (let say, more than 5) ?
1. does it only apply for certain root?
2. does it apply for large period, or only temporary?

In [38]:
root_list = 'acdfijklm'

for root in root_list:
    for item in anycast['{}_root'.format(root)].find({'$where': 'function() { if(this.path4.length > 5) {return true;} else { return false; } }'}):
        dt = datetime.fromtimestamp(item['timestamp']).strftime('%d/%m/%y')
        print('{}-Root [{}] {}:\t{}\t{}'.format(root, dt, item['peer'], len(item['path4']), len(item['path6'])))

a-Root [01/07/09] 196613:	6	5
a-Root [01/07/09] 29449:	6	4
a-Root [01/07/09] 1930:	6	5
a-Root [01/07/09] 20640:	7	3
a-Root [01/07/09] 6881:	6	5
a-Root [01/08/09] 196613:	6	6
a-Root [01/08/09] 20640:	7	3
a-Root [01/09/09] 196613:	6	5
a-Root [01/09/09] 20640:	7	3
a-Root [01/09/09] 6881:	6	5
a-Root [01/10/09] 196613:	6	5
a-Root [01/10/09] 20640:	7	3
a-Root [01/10/09] 6881:	6	5
a-Root [01/11/09] 196613:	6	5
a-Root [01/11/09] 20640:	7	3
a-Root [01/12/09] 20640:	7	3
a-Root [01/12/09] 6881:	6	5
a-Root [01/02/10] 196613:	6	5
a-Root [01/03/10] 196613:	7	5
a-Root [01/04/10] 196613:	6	5
a-Root [01/05/10] 196613:	6	5
a-Root [01/06/10] 196613:	6	5
a-Root [01/07/10] 196613:	6	5
a-Root [01/07/10] 22548:	6	4
a-Root [01/08/10] 196613:	6	5
a-Root [01/08/10] 22548:	6	4
a-Root [01/09/10] 196613:	6	5
a-Root [01/09/10] 22548:	6	4
a-Root [01/10/10] 196613:	6	5
a-Root [01/11/10] 196613:	7	5
a-Root [01/11/10] 6881:	6	5
a-Root [01/12/10] 196613:	7	5
a-Root [01/01/11] 196613:	6	5
a-Root [01/02/11] 28289:	6	6
a-R

In [34]:
unique_list = set()
result = dict()
for i in range(6,10):
    result[i] = []
    print('*****************************************************************')
    print('Hop: {}'.format(i))
    print('*****************************************************************')
    for root in root_list:
        res = anycast['{}_root'.format(root)].aggregate(
            [
                {'$match': {'path4': {'$size': i } }},
                {'$group': {"_id": "$peer", "count": {"$sum": 1} }},
                {'$sort': {'count': -1}}
            ])
        print('{}-Root'.format(root))
        for item in res:
            print(item)
            result[i].append({'peer': item['_id'], 'count': item['count'], 'root': root})
            unique_list.add(item['_id'])
        print('====')

*****************************************************************
Hop: 6
*****************************************************************
a-Root
{'_id': 14840, 'count': 18}
{'_id': 28289, 'count': 17}
{'_id': 196613, 'count': 16}
{'_id': 22548, 'count': 16}
{'_id': 28138, 'count': 8}
{'_id': 57821, 'count': 7}
{'_id': 57118, 'count': 5}
{'_id': 49605, 'count': 5}
{'_id': 6881, 'count': 5}
{'_id': 15469, 'count': 4}
{'_id': 263584, 'count': 2}
{'_id': 1930, 'count': 2}
{'_id': 6679, 'count': 1}
{'_id': 29449, 'count': 1}
====
c-Root
{'_id': 50763, 'count': 7}
{'_id': 52888, 'count': 3}
====
d-Root
{'_id': 14840, 'count': 20}
{'_id': 28289, 'count': 9}
{'_id': 553, 'count': 8}
{'_id': 53070, 'count': 8}
{'_id': 37989, 'count': 7}
{'_id': 52720, 'count': 3}
{'_id': 263584, 'count': 2}
{'_id': 262317, 'count': 1}
{'_id': 22548, 'count': 1}
====
f-Root
{'_id': 35369, 'count': 15}
{'_id': 14840, 'count': 13}
{'_id': 28289, 'count': 8}
{'_id': 1853, 'count': 7}
{'_id': 57118, 'count': 5}
{'_

In [35]:
result

{6: [{'count': 18, 'peer': 14840, 'root': 'a'},
  {'count': 17, 'peer': 28289, 'root': 'a'},
  {'count': 16, 'peer': 196613, 'root': 'a'},
  {'count': 16, 'peer': 22548, 'root': 'a'},
  {'count': 8, 'peer': 28138, 'root': 'a'},
  {'count': 7, 'peer': 57821, 'root': 'a'},
  {'count': 5, 'peer': 57118, 'root': 'a'},
  {'count': 5, 'peer': 49605, 'root': 'a'},
  {'count': 5, 'peer': 6881, 'root': 'a'},
  {'count': 4, 'peer': 15469, 'root': 'a'},
  {'count': 2, 'peer': 263584, 'root': 'a'},
  {'count': 2, 'peer': 1930, 'root': 'a'},
  {'count': 1, 'peer': 6679, 'root': 'a'},
  {'count': 1, 'peer': 29449, 'root': 'a'},
  {'count': 7, 'peer': 50763, 'root': 'c'},
  {'count': 3, 'peer': 52888, 'root': 'c'},
  {'count': 20, 'peer': 14840, 'root': 'd'},
  {'count': 9, 'peer': 28289, 'root': 'd'},
  {'count': 8, 'peer': 553, 'root': 'd'},
  {'count': 8, 'peer': 53070, 'root': 'd'},
  {'count': 7, 'peer': 37989, 'root': 'd'},
  {'count': 3, 'peer': 52720, 'root': 'd'},
  {'count': 2, 'peer': 2635

In [36]:
unique_list

{251,
 286,
 513,
 553,
 559,
 680,
 1103,
 1853,
 1916,
 1930,
 3257,
 6679,
 6762,
 6850,
 6881,
 6939,
 7575,
 8426,
 8596,
 8758,
 9189,
 13030,
 14840,
 15469,
 20640,
 20764,
 22548,
 22652,
 25091,
 25152,
 25160,
 28138,
 28289,
 28329,
 28917,
 29140,
 29449,
 29608,
 29636,
 30132,
 31019,
 34225,
 34695,
 35369,
 37989,
 39912,
 47872,
 49605,
 50763,
 52720,
 52888,
 53070,
 56730,
 57118,
 57381,
 57821,
 196613,
 262317,
 263584}

In [39]:
from pandas import DataFrame
import pandas as pd
import numpy as np